<a href="https://colab.research.google.com/github/MUNEEB68/Machine-learning-task/blob/main/linearregression_withoutneuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
from google.colab import files
from sklearn.preprocessing import StandardScaler

# Upload the CSV file
uploaded = files.upload()

# Load the CSV file into a pandas DataFrame
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# Display the first few rows of the DataFrame to ensure it loaded correctly
print(df.head())

# Check for missing values
print(df.isnull().sum())

# One-hot encode the 'Neighborhood' column
df = pd.get_dummies(df, columns=['Neighborhood'], drop_first=True)

# Extract input features and target variable
X = df.drop('Price', axis=1).values
y = df['Price'].values

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to torch tensors
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Initialize parameters
w_init = torch.zeros(X_train.size(1), 1, dtype=torch.float32)
b_init = torch.tensor(0.0, dtype=torch.float32)

# Function to calculate the cost
def compute_cost(X, y, w, b):
    m = X.size(0)
    predictions = X @ w + b
    cost = torch.sum((predictions - y) ** 2) / (2 * m)
    return cost

# Function to calculate the gradient
def compute_gradient(X, y, w, b):
    m = X.size(0)
    predictions = X @ w + b
    error = predictions - y
    dj_dw = (X.t() @ error) / m
    dj_db = torch.sum(error) / m
    return dj_dw, dj_db

# Function to perform gradient descent
def gradient_descent(X, y, w_in, b_in, alpha, num_iters, cost_function, gradient_function):
    w = w_in.clone().detach().requires_grad_(False)
    b = b_in.clone().detach().requires_grad_(False)

    J_history = []
    p_history = []

    for i in range(num_iters):
        dj_dw, dj_db = gradient_function(X, y, w, b)

        b -= alpha * dj_db
        w -= alpha * dj_dw

        if i < 100000:
            J_history.append(cost_function(X, y, w, b).item())
            p_history.append((w.clone(), b.item()))

        if i % (num_iters // 10) == 0:
            print(f"Iteration {i:4}: Cost {J_history[-1]:0.2e} ",
                  f"dj_dw: {dj_dw.norm(): 0.3e}, dj_db: {dj_db.item(): 0.3e}  ",
                  f"w: {w.norm(): 0.3e}, b: {b.item(): 0.5e}")

    return w, b, J_history, p_history

# Some gradient descent settings
iterations = 10000
alpha = 1.0e-3  # Reduce the learning rate

# Run gradient descent
w_final, b_final, J_hist, p_hist = gradient_descent(X_train, y_train, w_init, b_init, alpha,
                                                    iterations, compute_cost, compute_gradient)

print(f"(w, b) found by gradient descent: (w = {w_final.flatten().tolist()}, b = {b_final.item():8.4f})")


Saving housing_price_dataset.csv to housing_price_dataset (1).csv
   SquareFeet  Bedrooms  Bathrooms Neighborhood  YearBuilt          Price
0        2126         4          1        Rural       1969  215355.283618
1        2459         3          2        Rural       1980  195014.221626
2        1860         2          1       Suburb       1970  306891.012076
3        2294         2          1        Urban       1996  206786.787153
4        2130         5          2       Suburb       2001  272436.239065
SquareFeet      0
Bedrooms        0
Bathrooms       0
Neighborhood    0
YearBuilt       0
Price           0
dtype: int64
Iteration    0: Cost 2.81e+10  dj_dw:  5.750e+04, dj_db: -2.248e+05   w:  5.750e+01, b:  2.24827e+02
Iteration 1000: Cost 4.88e+09  dj_dw:  2.114e+04, dj_db: -8.267e+04   w:  3.637e+04, b:  1.42242e+05
Iteration 2000: Cost 1.74e+09  dj_dw:  7.778e+03, dj_db: -3.040e+04   w:  4.973e+04, b:  1.94461e+05
Iteration 3000: Cost 1.31e+09  dj_dw:  2.863e+03, dj_db: -1.118e+0